In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import requests

In [19]:
# URL of the page
url = "https://www.makemytrip.com/hotels/hotel-listing/?checkin=05272024&city=CTGOI&checkout=05302024&roomStayQualifier=1e0e&locusId=CTGOI&country=IN&locusType=city&searchText=Goa&regionNearByExp=3&rsc=1e2eundefinede"
# Configure Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0")
driver = webdriver.Chrome(options=options)

# Load the page
driver.get(url)

# Function to scroll down multiple times until all hotel names are loaded
def scroll_down():
    # Scroll down the page multiple times
    for _ in range(100): 
        # Scroll down to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait for a short while to allow content to load
        time.sleep(2)

# Scroll down to load more content
scroll_down()


soup = BeautifulSoup(driver.page_source, 'html.parser')

#hotel_names
hotel_names = soup.find_all("span", class_="wordBreak appendRight10")

Hotel_Name = []

for name in hotel_names:
    a = name.text
    Hotel_Name.append(a.split(",")[0])

#Reviews
review = soup.find_all("span",class_ = "ratingText latoBlack appendLeft3 darkBlueText font16")

Reviews = []

for Review in review:
    Reviews.append(Review.text)
    
num_to_add = len(Hotel_Name) - len(Reviews)
for i in range(num_to_add):
    Reviews.append(np.nan)
    
    

#Ratings
rating = soup.find_all("span",itemprop="ratingValue")

Ratings = []
for i in rating:
    Ratings.append(i.text)
    
Ratings = [i for i in Ratings if i != ""]
a = len(Hotel_Name) - len(Ratings)
Ratings += [np.nan] * a
    
#Number of Ratings
num_of_ratings = soup.find_all("p",class_ = "font14 darkGreyText appendTop5")

Number_Of_Ratings = []

for i in num_of_ratings:
    a = i.text.strip()
    if a:  # Check if 'a' is not an empty string
        b = re.findall(r"\d+", a)
        if b:  # Check if 'b' is not an empty list after finding digits
            Number_Of_Ratings.append(int(b[0]))  # Assuming you want the first number found
        else:
            Number_Of_Ratings.append(np.nan)  # Append np.nan if no digits are found
    else:
        Number_Of_Ratings.append(np.nan)  # Append np.nan for empty strings

# If the length of Number_Of_Ratings is less than 781, fill remaining values with np.nan
Number_Of_Ratings += [np.nan] * (len(Hotel_Name) - len(Number_Of_Ratings))

#cost of room
cost_of_room = soup.find_all("p",class_ = "priceText latoBlack font22 blackText appendBottom5")

Cost_Of_Room = []
for i in cost_of_room:
    Cost_Of_Room.append(i.text)

#Hotel Location
location = soup.find_all("span",class_ = "blueText")
 
Hotel_Location = []
for i in location[3:]:
    Hotel_Location.append(i.text)

#near by beaches

near_by_beaches = soup.find_all("span",class_ = "latoRegular")

beach = re.compile(r'.*beach.*', re.IGNORECASE)
beaches = [i.text for i in near_by_beaches if beach.match(i.text)]

Near_By_Beaches = []

for i in beaches:
    a = i.strip()
    if a:  # Check if 'a' is not an empty string
        b = re.findall(r"\b(\w+) Beach\b", a, re.IGNORECASE)
        if b:  # Check if 'b' is not an empty list after finding words
            Near_By_Beaches.append(b[0].strip())  # Append the first word found after stripping
        else:
            Near_By_Beaches.append(np.nan)  # Append np.nan if no words are found
    else:
        Near_By_Beaches.append(np.nan)  # Append np.nan for empty strings

# If the length of Near_By_Beaches is less than 781, fill remaining values with np.nan
Near_By_Beaches += [np.nan] * (len(Hotel_Name) - len(Near_By_Beaches))


# Close the WebDriver
driver.quit()


In [20]:
print(len(Hotel_Name))
print(len(Reviews))
print(len(Ratings))
print(len(Number_Of_Ratings))
print(len(Cost_Of_Room))
print(len(Hotel_Location))
print(len(Near_By_Beaches))

878
878
878
878
878
878
878


In [29]:
a = {"Hotel_Name":Hotel_Name,
     "Reviews":Reviews,
     "Ratings":Ratings,
    "Number_Of_Ratings":Number_Of_Ratings,
    "Cost_Of_Room":Cost_Of_Room,
    "Hotel_Location":Hotel_Location,
    "Near_By_Beaches":Near_By_Beaches
    }

In [31]:
df=pd.DataFrame(a)

In [32]:
df.to_csv(r"C:\Users\aresa\Goa_Hotels.csv",index = False)

In [33]:
df = pd.read_csv(r"C:\Users\aresa\Goa_Hotels.csv")

In [34]:
df

Hotel_Name     Reviews  Ratings  \
0                     Taj Exotica Resort & Spa   Excellent      4.8   
1                        Caravela Beach Resort   Excellent      4.3   
2    The Astor - All Suites Hotel Candolim Goa   Excellent      4.8   
3             Country Inn & Suites by Radisson   Very Good      3.8   
4                 Taj Fort Aguada Resort & Spa   Excellent      4.6   
..                                         ...         ...      ...   
873                          SPACELAND Mandrem         NaN      NaN   
874                              Rivasa Resort         NaN      NaN   
875                               Laxmi empire         NaN      NaN   
876     Coconut Creek Resort - Dabolim Airport         NaN      NaN   
877                                   Hotel AZ         NaN      NaN   

     Number_Of_Ratings Cost_Of_Room       Hotel_Location Near_By_Beaches  
0                995.0     ₹ 26,667             Benaulim        Benaulim  
1               4153.0     ₹ 10,499          Varca Beach           Varca  
2                 30.0      ₹ 8,181             Candolim        Candolim  
3               3163.0      ₹ 5,000             Candolim        Candolim  
4               2649.0     ₹ 18,833             Candolim      Sinquerium  
..                 ...          ...                  ...             ...  
873                NaN        ₹ 529        Mandrem Beach             NaN  
874                NaN      ₹ 1,316  Calangute Baga Road             NaN  
875                NaN      ₹ 3,178               Siolim             NaN  
876                NaN      ₹ 8,800        Bogmalo Beach             NaN  
877                NaN        ₹ 608             Candolim             NaN  

[878 rows x 7 columns]